In [ ]:
%pip install google-generativeai pdf2image pytesseract pillow pdfplumber

In [7]:
import os
import json
import time
from pathlib import Path
from datetime import datetime
from typing import Dict, Optional, Tuple
from dotenv import load_dotenv
from jsonschema import validate, ValidationError
from supabase import create_client, Client

print("✓ All libraries imported")

✓ All libraries imported


In [8]:
PDF_DIR = r"Z:\Risper M\2025 valuation reports\Year 2025"
OUTPUT_DIR = "./additional_2025v1"
ERROR_LOG_DIR = "./errors"

# Load environment variables
dotenv_path = r"C:\Users\samue\Documents\Work\Code\valuation_data_miner\.env"
load_dotenv(dotenv_path)

# API Keys
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
# print(f"Key: {SUPABASE_KEY}, \n URL: {SUPABASE_URL}")

# Model settings
GEMINI_MODEL = "models/gemini-2.5-flash" 
TESSERACT_PATH = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Create directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(ERROR_LOG_DIR, exist_ok=True)

print("✓ Configuration loaded")
print(f"  PDF Directory: {PDF_DIR}")
print(f"  Gemini Model: {GEMINI_MODEL}")

if not GEMINI_API_KEY:
    print("\n⚠ WARNING: GEMINI_API_KEY not found!")
    print("  Get key from: https://makersuite.google.com/app/apikey")

✓ Configuration loaded
  PDF Directory: Z:\Risper M\2025 valuation reports\Year 2025
  Gemini Model: models/gemini-2.5-flash


In [ ]:
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content("Hello, respond with a JSON: {\"test\": 123}", 
                                  generation_config=genai.types.GenerationConfig(
                                      temperature=0.1,
                                      max_output_tokens=200
                                  ))

print(response.text)


In [9]:
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)

models = genai.list_models()
for m in models:
    print(m.name)


InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

In [ ]:
VALUATION_SCHEMA = {
    "type": "object",
    "required": [
        "property_id",
        "report_reference",
        "market_value_amount",
        "metadata"
    ],
    "properties": {

        # --- CORE IDENTIFIERS ---
        "property_id": {"type": "string"},
        "report_reference": {"type": "string"},

        # --- TITLE INFORMATION ---
        "title_number": {"type": "string"},
        "lr_number": {"type": "string"},
        "ir_number": {"type": "string"},

        # --- CLIENT + VALUER ---
        "client_name": {"type": "string"},
        "valuer_name": {"type": "string"},
        "inspection_date": {"type": "string"},
        "valuation_date": {"type": "string"},

        # --- LOCATION ---
        "location_county": {"type": "string"},
        "location_description": {"type": "string"},
        "location_coordinates": {"type": "string"},

        # --- PARCEL DETAILS ---
        "plot_area_hectares": {"type": "number"},
        "plot_area_acres": {"type": "number"},
        "land_use": {"type": "string"},

        # --- NEW LAND PHYSICAL FEATURES ---
        "plot_shape": {"type": "string"},     # rectangular, irregular, square
        "soil_type": {"type": "string"},      # black cotton, red soil, sandy, loam, murram
        "gradient": {"type": "string"},       # flat, gentle slope, moderate slope, steep
        "drainage": {"type": "string"},       # optional recommendation
        "vegetation": {"type": "string"},     # optional recommendation

        # --- TENURE + OWNERSHIP ---
        "tenure_type": {"type": "string"},
        "registered_proprietor": {"type": "string"},
        "ownership_type": {"type": "string"},
        "encumbrances": {"type": "string"},

        # --- VALUATION ---
        "market_value_amount": {"type": "number"},
        "market_value_currency": {"type": "string"},

        # --- METADATA ---
        "metadata": {
            "type": "object",
            "properties": {
                "source_file": {"type": "string"},
                "file_size_kb": {"type": "number"},
                "processing_time_seconds": {"type": "number"},
                "ocr_used": {"type": "boolean"},
                "pages_processed": {"type": "number"},
                "model_name": {"type": "string"},
                "timestamp": {"type": "string"}
            }
        }
    }
}

print("✓ Updated JSON schema defined")


In [ ]:
def validate_extracted_data(data: Dict) -> Tuple[bool, Optional[str]]:
    """Validate extracted data against schema"""
    try:
        validate(instance=data, schema=VALUATION_SCHEMA)
        return True, None
    except ValidationError as e:
        return False, str(e)

print("✓ Validation function defined")


In [ ]:
def init_supabase() -> Optional[Client]:
    """Initialize Supabase client"""
    try:
        if not SUPABASE_URL or not SUPABASE_KEY:
            print("⚠ Supabase credentials not configured")
            return None
        
        client = create_client(SUPABASE_URL, SUPABASE_KEY)
        print("✓ Supabase client initialized")
        return client
    except Exception as e:
        print(f"✗ Supabase init failed: {e}")
        return None

supabase_client = init_supabase()


In [ ]:
SUPABASE_TABLE = "nrb_2025"   


def sanitize_dates_for_supabase(data: dict) -> dict:
    """Convert empty string dates to None for Supabase insert"""
    date_fields = ["inspection_date", "valuation_date"]
    for field in date_fields:
        if field in data and (data[field] == "" or data[field] is None):
            data[field] = None
    return data


def upload_to_supabase(client: Client, data: Dict, filename: str) -> bool:
    """Upload extracted valuation data to Supabase (restricted to valid columns)."""
    if client is None:
        return False
    
    try:
        upload_data = sanitize_dates_for_supabase(data.copy())

        # Allowed columns EXACTLY as in your Supabase table
        allowed_columns = {
            "property_id",
            "report_reference",
            "title_number",
            "lr_number",
            "ir_number",
            "client_name",
            "valuer_name",
            "inspection_date",
            "valuation_date",
            "location_county",
            "location_description",
            "location_coordinates",
            "plot_area_hectares",
            "plot_area_acres",
            "land_use",
            "plot_shape",
            "soil_type",
            "gradient",
            "drainage",
            "vegetation",
            "tenure_type",
            "registered_proprietor",
            "ownership_type",
            "encumbrances",
            "market_value_amount",
            "market_value_currency",
            "metadata"
        }

        # Remove any field not in table
        clean_record = {k: v for k, v in upload_data.items() if k in allowed_columns}

        # Insert into your table
        client.table(SUPABASE_TABLE).insert(clean_record).execute()

        return True
    
    except Exception as e:
        raise Exception(f"Supabase upload failed: {str(e)}")


In [ ]:
def process_scanned_pdf_with_ocr(pdf_path: str, dpi: int = 150) -> str:
    """Convert scanned PDF to text using Tesseract OCR safely (memory-friendly)."""
    try:
        from pdf2image import convert_from_path
        import pytesseract
        import gc

        # Set Tesseract path
        if os.path.exists(TESSERACT_PATH):
            pytesseract.pytesseract.tesseract_cmd = TESSERACT_PATH
        else:
            raise Exception(f"Tesseract not found at {TESSERACT_PATH}. Install from: https://github.com/UB-Mannheim/tesseract/wiki")
        
        print(f"    → Converting PDF to images at {dpi} dpi...")
        
        # Convert PDF to images
        images = convert_from_path(pdf_path, dpi=dpi, fmt='jpeg', thread_count=2)
        print(f"    → Running OCR on {len(images)} pages...")
        
        all_text = []
        for i, image in enumerate(images):
            print(f"      → Page {i+1}/{len(images)}...", end=' ')
            
            # OCR the page (use psm 6 for memory efficiency)
            text = pytesseract.image_to_string(
                image,
                lang='eng',
                config='--psm 6'
            ).strip()
            
            if text:
                all_text.append(f"--- Page {i+1} ---\n{text}")
                print(f"✓ {len(text)} chars")
            else:
                print("(empty)")
            
            # Free memory after each page
            image.close()
            del image
            gc.collect()
        
        full_text = "\n\n".join(all_text)
        print(f"    → OCR complete: {len(full_text)} characters")
        
        # Save OCR output for debugging
        ocr_file = os.path.join(ERROR_LOG_DIR, f"{Path(pdf_path).stem}_ocr_text.txt")
        with open(ocr_file, 'w', encoding='utf-8') as f:
            f.write(full_text)
        
        return full_text
        
    except Exception as e:
        raise Exception(f"OCR failed: {str(e)}")

print("✓ Updated OCR function defined")


In [ ]:
def process_pdf_with_fallback(pdf_path: str) -> str:
    """Try multiple extraction methods"""
    
    # Try pdfplumber first
    try:
        import pdfplumber
        print(f"    → Trying pdfplumber...")
        
        all_text = []
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text:
                    all_text.append(f"--- Page {i+1} ---\n{text}")
        
        full_text = "\n\n".join(all_text)
        
        if len(full_text) > 500:
            print(f"    ✓ pdfplumber: {len(full_text)} chars")
            return full_text
        else:
            print(f"    → Only {len(full_text)} chars, trying OCR...")
    except Exception as e:
        print(f"    → pdfplumber failed, trying OCR...")
    
    # Use OCR
    return process_scanned_pdf_with_ocr(pdf_path)

print("✓ PDF extraction with fallback defined")

In [ ]:
import re
import time
import json
from pathlib import Path
import google.generativeai as genai
from time import sleep

# -----------------------------
# Helper functions
# -----------------------------
def clean_text_for_model(text: str) -> str:
    """Remove page headers, footers, and empty lines"""
    text = re.sub(r'--- Page \d+ ---', '', text)
    text = re.sub(r'\n\s*\n', '\n', text)
    return text.strip()

def find_report_reference(text: str) -> str:
    """Detect report reference numbers"""
    pattern = r"\b(?:[A-Z]{2,3}/)?(?:[A-Z]{2,3}/)?\d{3,5}/\d{1,2}/\d{2,4}\b"
    matches = re.findall(pattern, text)
    if matches:
        return max(matches, key=len)
    return ""

def clean_gemini_json(response_text: str) -> str:
    """Extract JSON from Gemini response"""
    if '```json' in response_text:
        response_text = response_text.split('```json')[1].split('```')[0]
    elif '```' in response_text:
        response_text = response_text.split('```')[1].split('```')[0]
    response_text = response_text.strip()
    start = response_text.find('{')
    end = response_text.rfind('}')
    if start != -1 and end != -1:
        response_text = response_text[start:end+1]
    return response_text

def deep_merge(d1, d2):
    """Merge dictionaries recursively"""
    for k, v in d2.items():
        if k in d1 and isinstance(d1[k], dict) and isinstance(v, dict):
            d1[k] = deep_merge(d1[k], v)
        else:
            d1[k] = v
    return d1

# -----------------------------
# CRITICAL: Configure safety settings
# -----------------------------
def get_safe_generation_config():
    """Returns generation config with safety settings disabled"""
    safety_settings = [
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_NONE"
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_NONE"
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_NONE"
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_NONE"
        }
    ]
    
    generation_config = genai.types.GenerationConfig(
        temperature=0.1,
        max_output_tokens=4096,
        response_mime_type="application/json"
    )
    
    return generation_config, safety_settings

# -----------------------------
# Rate limiting with retry logic
# -----------------------------
def call_gemini_with_retry(model, prompt, generation_config, safety_settings, max_retries=3):
    """
    Call Gemini API with automatic retry on rate limit errors.
    Implements exponential backoff.
    """
    for attempt in range(max_retries):
        try:
            response = model.generate_content(
                prompt,
                generation_config=generation_config,
                safety_settings=safety_settings
            )
            
            # Check if response was blocked
            if response.prompt_feedback.block_reason:
                print(f"      ⚠ Response blocked: {response.prompt_feedback.block_reason}")
                return None
            
            # Check if we have valid text
            if hasattr(response, 'text'):
                return response
            else:
                print(f"      ⚠ No text in response")
                return None
                
        except Exception as e:
            error_str = str(e)
            
            # Check if it's a rate limit error (429)
            if "429" in error_str or "quota" in error_str.lower():
                # Extract retry delay from error message
                import re
                retry_match = re.search(r'retry in (\d+\.\d+)s', error_str)
                
                if retry_match:
                    retry_seconds = float(retry_match.group(1))
                else:
                    # Default exponential backoff: 5s, 15s, 30s
                    retry_seconds = 5 * (3 ** attempt)
                
                if attempt < max_retries - 1:
                    print(f"      ⚠ Rate limit hit. Waiting {retry_seconds:.1f}s before retry {attempt+1}/{max_retries}...")
                    sleep(retry_seconds)
                else:
                    print(f"      ✗ Rate limit exceeded after {max_retries} attempts")
                    raise e
            else:
                # Non-rate-limit error, raise immediately
                raise e
    
    return None

# -----------------------------
# Extraction function with rate limiting
# -----------------------------
def extract_with_gemini(text_content: str, filename: str) -> dict:
    """
    Extract structured property valuation data from a Kenyan PDF OCR text.
    Two-pass approach with automatic rate limiting.
    """
    
    combined_result = {}
    cleaned_text = clean_text_for_model(text_content)
    report_ref_guess = find_report_reference(cleaned_text[:3000])
    
    # Get safe configuration
    generation_config, safety_settings = get_safe_generation_config()
    
    # Initialize model with safety settings
    # Using gemini-2.0-flash instead of 2.0 for better rate limits
    model = genai.GenerativeModel(
        "models/gemini-2.0-flash",  # Changed to 1.5 for better free tier limits
        safety_settings=safety_settings
    )
    
    # -----------------------------
    # PASS 1: Core identifiers
    # -----------------------------
    prompt_ids = f"""
You are performing a DOCUMENT TRANSCRIPTION task.
Extract structured data EXACTLY as it appears in the source document.

Extract these fields from the Kenyan valuation report:
- report_reference (format: SOO/DOO/5310/1/25)
- title_number
- lr_number
- ir_number
- client_name
- valuer_name
- inspection_date (YYYY/MM/DD format)
- valuation_date (YYYY/MM/DD format)

Hint for report_reference: "{report_ref_guess}"

Return ONLY this JSON structure:
{{
  "property_id": "",
  "report_reference": "",
  "title_number": "",
  "lr_number": "",
  "ir_number": "",
  "client_name": "",
  "valuer_name": "",
  "inspection_date": "",
  "valuation_date": ""
}}

SOURCE TEXT (first 5000 chars):
{cleaned_text[:5000]}
"""
    
    try:
        print("      → Extracting IDs...")
        response = call_gemini_with_retry(model, prompt_ids, generation_config, safety_settings)
        
        if response:
            ids_json = clean_gemini_json(response.text)
            extracted_ids = json.loads(ids_json)
            combined_result = deep_merge(combined_result, extracted_ids)
            print("      ✓ IDs extracted")
        else:
            print("      ⚠ ID extraction failed")
            
    except json.JSONDecodeError as e:
        print(f"      ⚠ JSON parsing error in IDs: {e}")
    except Exception as e:
        print(f"      ⚠ Gemini ID extraction failed: {e}")
    
    # Add delay between API calls to respect rate limits
    sleep(2)  # 2 second delay between passes
    
    # -----------------------------
    # PASS 2: Land, valuation, metadata
    # -----------------------------
    prompt_land = f"""
You are performing a DOCUMENT TRANSCRIPTION task.
Extract structured data EXACTLY as it appears in the source document.

Extract these fields from the Kenyan valuation report:
- location_county
- location_description
- location_coordinates
- plot_area_hectares (number)
- plot_area_acres (number)
- land_use
- plot_shape
- soil_type
- gradient
- tenure_type
- registered_proprietor
- encumbrances
- market_value_amount (number)
- market_value_currency

Return ONLY this JSON structure:
{{
  "location_county": "",
  "location_description": "",
  "location_coordinates": "",
  "plot_area_hectares": 0,
  "plot_area_acres": 0,
  "land_use": "",
  "plot_shape": "",
  "soil_type": "",
  "gradient": "",
  "drainage": "",
  "vegetation": "",
  "tenure_type": "",
  "registered_proprietor": "",
  "ownership_type": "",
  "encumbrances": "",
  "market_value_amount": 0,
  "market_value_currency": "KShs"
}}

SOURCE TEXT:
{cleaned_text[:15000]}
"""
    
    try:
        print("      → Extracting land/valuation data...")
        response = call_gemini_with_retry(model, prompt_land, generation_config, safety_settings)
        
        if response:
            land_json = clean_gemini_json(response.text)
            extracted_land = json.loads(land_json)
            combined_result = deep_merge(combined_result, extracted_land)
            print("      ✓ Land data extracted")
        else:
            print("      ⚠ Land extraction failed")
            
    except json.JSONDecodeError as e:
        print(f"      ⚠ JSON parsing error in Land: {e}")
    except Exception as e:
        print(f"      ⚠ Gemini Land extraction failed: {e}")
    
    # -----------------------------
    # Final adjustments
    # -----------------------------
    if not combined_result.get("property_id"):
        combined_result["property_id"] = Path(filename).stem
    if "report_reference" not in combined_result or not combined_result["report_reference"]:
        combined_result["report_reference"] = report_ref_guess
    
    # Metadata skeleton
    if "metadata" not in combined_result:
        combined_result["metadata"] = {
            "source_file": filename,
            "file_size_kb": 0,
            "processing_time_seconds": 0,
            "ocr_used": True,
            "pages_processed": 0,
            "model_name": "gemini-1.5-flash",
            "timestamp": ""
        }
    
    # Ensure string fields
    string_fields = [
        "title_number", "lr_number", "ir_number", "client_name",
        "valuer_name", "inspection_date", "valuation_date"
    ]
    
    for field in string_fields:
        if field in combined_result and combined_result[field] is None:
            combined_result[field] = ""
    
    return combined_result

In [ ]:
def process_single_pdf_complete(pdf_path: str, filename: str) -> dict:
    """Complete pipeline: OCR → Gemini → Validate → Save"""
    result = {
        'filename': filename,
        'success': False,
        'stage': None,
        'error': None,
        'data': None,
        'timing': {}
    }
    
    start_time = time.time()
    
    try:
        # Stage 1: OCR
        result['stage'] = 'ocr'
        print(f"\n  📄 Stage 1: OCR")
        stage_start = time.time()
        
        text_content = process_pdf_with_fallback(pdf_path)
        pages_processed = text_content.count('--- Page')  # approximate page count
        result['timing']['ocr'] = time.time() - stage_start
        
        if len(text_content) < 200:
            raise Exception(f"Insufficient text: {len(text_content)} chars")
        
        print(f"  ✓ {len(text_content)} chars in {result['timing']['ocr']:.1f}s")
        
        # Stage 2: Gemini
        result['stage'] = 'gemini'
        print(f"\n  🤖 Stage 2: Gemini")
        stage_start = time.time()
        
        extracted_data = extract_with_gemini(text_content, filename)
        result['timing']['gemini'] = time.time() - stage_start
        result['data'] = extracted_data
        
        print(f"  ✓ Extracted in {result['timing']['gemini']:.1f}s")
        
        # -------------------------
        # Inject real metadata
        # -------------------------
        file_size_kb = round(Path(pdf_path).stat().st_size / 1024, 2)
        processing_time = round(time.time() - start_time, 2)
        
        if "metadata" not in extracted_data:
            extracted_data["metadata"] = {}
        
        extracted_data["metadata"].update({
            "source_file": filename,
            "file_size_kb": file_size_kb,
            "processing_time_seconds": processing_time,
            "ocr_used": True,
            "pages_processed": pages_processed,
            "model_name": GEMINI_MODEL,
            "timestamp": datetime.now().isoformat()
        })
        
        # Stage 3: Validation
        result['stage'] = 'validation'
        print(f"\n  ✅ Stage 3: Validation")
        
        is_valid, validation_error = validate_extracted_data(extracted_data)
        if not is_valid:
            print(f"  ⚠ Warning: {validation_error[:100]}")
        else:
            print(f"  ✓ Valid")
        
        # Stage 4: Save
        result['stage'] = 'saving'
        print(f"\n  💾 Stage 4: Save")
        output_file = os.path.join(OUTPUT_DIR, f"{Path(filename).stem}.json")
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(extracted_data, f, indent=2, ensure_ascii=False)
        print(f"  ✓ Saved to {Path(output_file).name}")
        
        # Stage 5: Supabase
        if supabase_client and is_valid:
            result['stage'] = 'supabase'
            try:
                upload_to_supabase(supabase_client, extracted_data, filename)
                print(f"  ✓ Uploaded to Supabase")
            except Exception as e:
                print(f"  ⚠ Supabase: {str(e)[:50]}")
        
        result['success'] = True
        result['timing']['total'] = time.time() - start_time
        
    except Exception as e:
        result['error'] = str(e)
        result['timing']['total'] = time.time() - start_time
        
        error_file = os.path.join(ERROR_LOG_DIR, f"{Path(filename).stem}_error.txt")
        with open(error_file, 'w', encoding='utf-8') as f:
            f.write(f"Stage: {result['stage']}\nError: {result['error']}\n")
        
        print(f"\n  ✗ Failed: {result['error'][:100]}")
    
    return result

print("✓ Complete pipeline defined with full metadata")


In [ ]:
def process_all_pdfs_complete():
    """Process all PDFs in PDF_DIR, generate OCR, send to Gemini, save JSON, and upload to Supabase.
       Skips PDFs that already have a JSON output file.
    """

    # -------------------------
    # Scan directories
    # -------------------------
    pdf_files = list(Path(PDF_DIR).glob("*.pdf"))
    if not pdf_files:
        print(f"No PDFs found in {PDF_DIR}")
        return []

    # Identify JSON files already processed
    processed_json_stems = {p.stem for p in Path(OUTPUT_DIR).glob("*.json")}

    # Filter: process ONLY new PDFs
    pdf_files = [p for p in pdf_files if p.stem not in processed_json_stems]

    if not pdf_files:
        print("🎉 All PDFs already processed — nothing to do.")
        return []

    print(f"\n{'='*70}")
    print(f"🚀 PROCESSING {len(pdf_files)} NEW PDFs")
    print(f"{'='*70}")

    results = []
    total_start = time.time()

    # -------------------------
    # Process each PDF
    # -------------------------
    for idx, pdf_path in enumerate(pdf_files, 1):

        print(f"\n{'='*70}")
        print(f"[{idx}/{len(pdf_files)}] {pdf_path.name}")
        print(f"{'='*70}")

        file_start_time = time.time()
        file_size_kb = round(pdf_path.stat().st_size / 1024, 2)

        # -------------------------
        # 1. OCR extraction
        # -------------------------
        text_content = process_pdf_with_fallback(str(pdf_path))
        pages_processed = text_content.count('--- Page')  # reliable heuristic
        print(f"  ✓ OCR complete ({pages_processed} pages, {len(text_content)} chars)")

        # -------------------------
        # 2. Gemini extraction
        # -------------------------
        extracted = extract_with_gemini(text_content, pdf_path.name)

        # -------------------------
        # 3. Add metadata to JSON
        # -------------------------
        processing_time = round(time.time() - file_start_time, 2)
        if "metadata" not in extracted:
            extracted["metadata"] = {}

        extracted["metadata"].update({
            "source_file": pdf_path.name,
            "file_size_kb": file_size_kb,
            "processing_time_seconds": processing_time,
            "ocr_used": True,
            "pages_processed": pages_processed,
            "model_name": GEMINI_MODEL,
            "timestamp": datetime.now().isoformat()
        })

        # -------------------------
        # 4. Save the JSON output
        # -------------------------
        output_path = os.path.join(OUTPUT_DIR, f"{pdf_path.stem}.json")
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(extracted, f, indent=2, ensure_ascii=False)

        print(f"✓ Saved → {output_path}")

        # -------------------------
        # 5. Upload to Supabase
        # -------------------------
        if supabase_client:
            try:
                upload_to_supabase(supabase_client, extracted, pdf_path.name)
                print(f"✓ Uploaded to Supabase")
            except Exception as e:
                print(f"⚠ Supabase upload failed: {str(e)[:120]}")

        # -------------------------
        # 6. Track results
        # -------------------------
        results.append({
            "filename": pdf_path.name,
            "success": bool(extracted),
            "data": extracted,
            "timing": {"processing_seconds": processing_time},
        })

    # -------------------------
    # Save batch summary
    # -------------------------
    total_time = round(time.time() - total_start, 2)
    summary_file = os.path.join(OUTPUT_DIR, "summary.json")

    with open(summary_file, "w", encoding="utf-8") as f:
        json.dump({
            "timestamp": datetime.now().isoformat(),
            "model": GEMINI_MODEL,
            "total_pdfs": len(pdf_files),
            "total_time_seconds": total_time,
            "results": results,
        }, f, indent=2)

    print(f"\n🎉 Batch finished in {total_time/60:.1f} minutes")
    return results


In [ ]:
def test_single_pdf():
    """Test with first PDF"""
    pdf_files = list(Path(PDF_DIR).glob("*.pdf"))
    if not pdf_files:
        print("No PDFs found")
        return
    
    test_file = pdf_files[0]
    print(f"\n🧪 TESTING: {test_file.name}\n")
    
    result = process_single_pdf_complete(str(test_file), test_file.name)
    
    if result['success']:
        print(f"\n✅ TEST PASSED!")
        print(f"\nPreview:")
        print(json.dumps(result['data'], indent=2)[:1000])
    else:
        print(f"\n❌ TEST FAILED")
    
    return result


print("\n" + "="*70)
print("="*70)
test_single_pdf()
# print("or:  results = process_all_pdfs_complete()")

In [ ]:
results = process_all_pdfs_complete()
results